In [1]:
!curl https://conceptnet.s3.amazonaws.com/downloads/2017/numberbatch/numberbatch-en-17.06.txt.gz --output numberbatch-en-17.06.txt.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  257M  100  257M    0     0  1955k      0  0:02:14  0:02:14 --:--:-- 2770k6k


https://github.com/loretoparisi/word2vec-twitter

In [5]:
#!gunzip numberbatch-en-17.06.txt.gz

In [376]:
!curl http://nlp.stanford.edu/data/wordvecs/glove.twitter.27B.zip --output glove.twitter.27B.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   326  100   326    0     0    765      0 --:--:-- --:--:-- --:--:--   765


In [68]:
!unzip crawl-300d-2M-subword.zip

Archive:  crawl-300d-2M-subword.zip
  inflating: crawl-300d-2M-subword.vec  
  inflating: crawl-300d-2M-subword.bin  


In [377]:
!unzip glove.twitter.27B.zip

Archive:  glove.twitter.27B.zip
  inflating: glove.twitter.27B.25d.txt  
  inflating: glove.twitter.27B.50d.txt  
  inflating: glove.twitter.27B.100d.txt  
  inflating: glove.twitter.27B.200d.txt  


In [67]:
!curl https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip --output crawl-300d-2M-subword.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5558M  100 5558M    0     0   9.8M      0  0:09:23  0:09:23 --:--:-- 7807k  3  211M    0     0  9059k      0  0:10:28  0:00:23  0:10:05  9.9M 5  301M    0     0  9390k      0  0:10:06  0:00:32  0:09:34 10.0M  0  9905k      0  0:09:34  0:01:16  0:08:18 10.0M:45  0:06:37 10.0M  0:09:20  0:03:42  0:05:38 10.0MM      0  0:09:19  0:04:34  0:04:45 10.0M3263M    0     0   9.9M      0  0:09:18  0:05:27  0:03:51 10.0MM  0  0:09:17  0:07:05  0:02:12 10.0MM   77 4310M    0     0   9.9M      0  0:09:16  0:07:11  0:02:05 10.0M   0   9.9M      0  0:09:16  0:07:14  0:02:02 10.1M9:18  0:09:04  0:00:14 7763k


In [2]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

#tmp_file = get_tmpfile("glove_word2vec.txt")
#_ = glove2word2vec('glove.twitter.27B.200d.txt', tmp_file)

#glove = KeyedVectors.load_word2vec_format(tmp_file)

In [2]:
glove.most_similar(['glove'])

[('gloves', 0.7430825233459473),
 ('helmet', 0.6035714149475098),
 ('cleats', 0.5549577474594116),
 ('shoe', 0.5447742938995361),
 ('boot', 0.5390670299530029),
 ('leather', 0.5314798355102539),
 ('hand', 0.5252668857574463),
 ('pads', 0.523605227470398),
 ('gear', 0.5217989683151245),
 ('coat', 0.5216744542121887)]

In [2]:
numberbatch = KeyedVectors.load_word2vec_format("numberbatch-en-17.06.txt.gz", binary=False)

In [3]:
numberbatch.most_similar(['functor'])

/home/szubovych/.virtualenvs/nlp/lib/python3.6/site-packages/gensim/models/keyedvectors.py:2356: RuntimeWarning: invalid value encountered in true_divide
  return (m / dist).astype(REAL)


[('contravariant_functor', 0.9674091935157776),
 ('forgetful_functor', 0.9666687250137329),
 ('yoneda_embedding', 0.9497337937355042),
 ('endofunctor', 0.9360368847846985),
 ('representable_functor', 0.9314213991165161),
 ('cofunctor', 0.9296932220458984),
 ('natural_transformation', 0.9164144992828369),
 ('yoneda_lemma', 0.9022417664527893),
 ('coaugmentation', 0.8871707320213318),
 ('category_theory', 0.8751257658004761)]

In [3]:
google = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)

In [5]:
google.most_similar(['apple'])

[('apples', 0.7203598022460938),
 ('pear', 0.6450696587562561),
 ('fruit', 0.6410146355628967),
 ('berry', 0.6302294731140137),
 ('pears', 0.6133961081504822),
 ('strawberry', 0.6058261394500732),
 ('peach', 0.6025873422622681),
 ('potato', 0.596093475818634),
 ('grape', 0.5935864448547363),
 ('blueberry', 0.5866668224334717)]

In [3]:
fasttext = KeyedVectors.load_word2vec_format("5d7b5525-b769-4579-9d6f-1aa9d8d1a1b1 ", binary=False)

In [4]:
from collections import namedtuple

Phrase = namedtuple('Phrase', 'original candidate label')
Token = namedtuple('Token', 'text tags')

def split_tokens(sent):
    tokens = []
    for token in sent.split():
        tags = token.split('/')
        tokens.append(Token(tags[0].lower(), tuple(tags[1:])))
    return tokens


def readData(filename, eval_label, ignoreNone):
    data = []
    with open(filename) as f:
        for line in f:
            fields = line.strip().split('\t')
            if len(fields) == 7:
                (trendid, trendname, origsent, candsent, judge, origsenttag, candsenttag) = fields
            else:
                continue
            label = eval_label(judge)
            if ((label is None) and ignoreNone):
                continue
            data.append(Phrase(split_tokens(origsenttag), split_tokens(candsenttag), label))
    
    return data
                
def eval_amt_label(label):
    nYes = eval(label)[0]            
    
    if nYes >= 3:
        return True
    elif nYes <= 1:
        return False
    
    return None

def eval_expert_label(label):
    nYes = int(label[0])
    
    if nYes >= 4:
        return True
    elif nYes <= 2:
        return False
    
    return None


def readTrainData(filename):
    return readData(filename, eval_amt_label, True)

def readTestData(filename):
    return readData(filename, eval_expert_label, False)

In [5]:
train_data = readTrainData("SemEval-PIT2015-py3/data/train.data")
dev_data = readTrainData("SemEval-PIT2015-py3/data/dev.data")
test_data = [p for p in readTestData("SemEval-PIT2015-py3/data/test.data") if p.label is not None]

In [6]:
len(train_data)

11530

In [7]:
len(test_data)

838

In [8]:
train_data[:3]

[Phrase(original=[Token(text='ej', tags=('B-person', 'NNP', 'B-NP', 'O')), Token(text='manuel', tags=('I-person', 'NNP', 'B-VP', 'O')), Token(text='the', tags=('O', 'DT', 'B-NP', 'O')), Token(text='1st', tags=('O', 'CD', 'I-NP', 'O')), Token(text='qb', tags=('O', 'NNP', 'I-NP', 'O')), Token(text='to', tags=('O', 'TO', 'B-VP', 'O')), Token(text='go', tags=('O', 'VB', 'I-VP', 'B-EVENT')), Token(text='in', tags=('O', 'IN', 'B-PP', 'I-EVENT')), Token(text='this', tags=('O', 'DT', 'B-NP', 'O')), Token(text='draft', tags=('O', 'NN', 'I-NP', 'O'))], candidate=[Token(text='but', tags=('O', 'CC', 'O', 'O')), Token(text='my', tags=('O', 'PRP$', 'B-NP', 'O')), Token(text='bro', tags=('O', 'NN', 'I-NP', 'O')), Token(text='from', tags=('O', 'IN', 'B-PP', 'O')), Token(text='the', tags=('O', 'DT', 'B-NP', 'O')), Token(text='757', tags=('O', 'CD', 'I-NP', 'O')), Token(text='ej', tags=('B-person', 'NNP', 'I-NP', 'O')), Token(text='manuel', tags=('I-person', 'NNP', 'I-NP', 'O')), Token(text='is', tags=(

In [9]:
test_data[:3]

[Phrase(original=[Token(text='all', tags=('O', 'DT', 'B-NP', 'O')), Token(text='the', tags=('O', 'DT', 'I-NP', 'O')), Token(text='home', tags=('O', 'NN', 'I-NP', 'O')), Token(text='alones', tags=('O', 'VBZ', 'B-VP', 'O')), Token(text='watching', tags=('O', 'VBG', 'I-VP', 'B-EVENT')), Token(text='8', tags=('O', 'CD', 'B-NP', 'O')), Token(text='mile', tags=('O', 'NN', 'I-NP', 'O'))], candidate=[Token(text='the', tags=('O', 'DT', 'B-NP', 'O')), Token(text='last', tags=('O', 'JJ', 'I-NP', 'O')), Token(text='rap', tags=('O', 'NN', 'I-NP', 'B-EVENT')), Token(text='battle', tags=('O', 'NN', 'I-NP', 'B-EVENT')), Token(text='in', tags=('O', 'IN', 'B-PP', 'O')), Token(text='8', tags=('O', 'CD', 'B-NP', 'O')), Token(text='mile', tags=('O', 'NNP', 'I-NP', 'O')), Token(text='nevr', tags=('O', 'NN', 'I-NP', 'O')), Token(text='gets', tags=('O', 'VBZ', 'B-VP', 'O')), Token(text='old', tags=('O', 'JJ', 'B-NP', 'O')), Token(text='ahah', tags=('O', 'JJ', 'I-NP', 'O'))], label=False),
 Phrase(original=[To

In [8]:
from nltk.corpus import stopwords
from tqdm import tqdm_notebook

NUM = set(['1st'])

def clean_sent(sent):
    new_sent = []
    for token in sent:
        if token.tags[0].startswith('B-'):
            new_sent.append(token.tags[0].split('-')[1])
            continue
        if token.tags[0].startswith('I-'): #or token.text in stopwords.words('english'):
            continue
        if token.tags[1] == 'CD' or token.text in NUM or token.text.isnumeric():
            new_sent.append('number')
            continue
        new_sent.append(token.text)
                            
    return new_sent

def clean_data(data):
    return [Phrase(clean_sent(phrase.original), clean_sent(phrase.candidate), phrase.label) \
            for phrase in tqdm_notebook(data)]

In [9]:
clean_train_data = clean_data(train_data)

In [10]:
clean_dev_data = clean_data(dev_data)

In [11]:
clean_test_data = clean_data(test_data)

In [12]:
clean_train_data[:3]

[Phrase(original=['person', 'the', 'number', 'qb', 'to', 'go', 'in', 'this', 'draft'], candidate=['but', 'my', 'bro', 'from', 'the', 'number', 'person', 'is', 'the', 'number', 'qb', 'gone'], label=True),
 Phrase(original=['person', 'the', 'number', 'qb', 'to', 'go', 'in', 'this', 'draft'], candidate=['can', 'believe', 'person', 'went', 'as', 'the', 'number', 'qb', 'in', 'the', 'draft'], label=True),
 Phrase(original=['person', 'the', 'number', 'qb', 'to', 'go', 'in', 'this', 'draft'], candidate=['person', 'is', 'the', 'number', 'qb', 'what'], label=True)]

In [13]:
from gensim.corpora import Dictionary

def phrase_tokens(phrase):
    return [token for token in (phrase.original + phrase.candidate)]
    

vocab = Dictionary([phrase_tokens(p) for p in clean_train_data + clean_dev_data + clean_test_data])

len(vocab)

8920

In [14]:
VOCAB_SIZE = len(vocab) + 1 # +1 for padding

print(VOCAB_SIZE)

8921


In [15]:
id2token = dict([(i, token)for token, i in vocab.token2id.items()])

In [16]:
def text_to_sequence(words):
    return [i + 1 for i in vocab.doc2idx(words)]

def sequence_to_text(seq):
    return [id2token[i - 1] for i in seq if i]

In [17]:
def data_to_sequences(data):
    
    encoder_seqs = []
    decoder_seqs = []
    labels = []
    
    for phrase in data:
        encoder_seqs.append(text_to_sequence([t for t in phrase.original]))
        decoder_seqs.append(text_to_sequence([t for t in phrase.candidate]))
        labels.append(phrase.label)
        
    return encoder_seqs, decoder_seqs, labels 

train_encoder_seqs, train_decoder_seqs, train_labels = data_to_sequences(clean_train_data)

print(len(train_encoder_seqs))
print(len(train_decoder_seqs))
print(len(train_labels))

11530
11530
11530


In [18]:
print(sequence_to_text(train_encoder_seqs[0]))
print(sequence_to_text(train_decoder_seqs[0]))
print(train_labels[0])

['person', 'the', 'number', 'qb', 'to', 'go', 'in', 'this', 'draft']
['but', 'my', 'bro', 'from', 'the', 'number', 'person', 'is', 'the', 'number', 'qb', 'gone']
True


In [19]:
dev_encoder_seqs, dev_decoder_seqs, dev_labels = data_to_sequences(clean_dev_data)

In [20]:
test_encoder_seqs, test_decoder_seqs, test_labels = data_to_sequences(clean_test_data)

In [ ]:
test_encoder_seqs, test_decoder_seqs, test_labels = data_to_sequences(clean_test_data)

In [21]:
MAX_SEQ_LEN = max([len(seq) for seq in (train_encoder_seqs + train_decoder_seqs + \
                                       dev_encoder_seqs + dev_decoder_seqs + \
                                       test_decoder_seqs + test_encoder_seqs)])

print(MAX_SEQ_LEN)

18


In [22]:
from keras.preprocessing.sequence import pad_sequences

def padding(sequences):
    return pad_sequences(sequences, maxlen=MAX_SEQ_LEN, dtype='int32', padding='post', truncating='post')

Using TensorFlow backend.


In [23]:
EMBEDDING_SIZE = 300

In [27]:
from nltk.stem.snowball import SnowballStemmer 

stemmer = SnowballStemmer('english')

In [24]:
import numpy as np

EMBEDDING_MATRIX = np.zeros((VOCAB_SIZE, EMBEDDING_SIZE))

def get_glove(word):
    try:
        return glove[word]
    except KeyError:
        pass
    return np.zeros(200)

def get_numberbatch(word):
    try:
        return numberbatch[word]
    except KeyError:
        pass
    return np.zeros(300)

missed = set([])

for word, i in vocab.token2id.items():   
    try:
        EMBEDDING_MATRIX[i] = fasttext[word]
    except KeyError:
        
        missed.add(word)

print(len(missed))

977


In [25]:
print(missed)

{'borussia', 'tasticdrews', 'cheick', 'leandowski', 'cowboysnation', 'ogmfmddkd', 'usain', 'sotheresthat', 'lovebenzox', 'yussss', 'ilyyyy', 'uuuuup', 'kcchiefs', 'subtchicago', 'oscarhe', 'mnwild', 'poppycorn', 'rbatto', 'chuddyer', 'theletout', 'relativeperspec', 'naterobinson', 'threepoint', 'jennabreanne13', 'cutondime25', 'muniscis', 'gundogan', 'xx11', 'holllllleeerrrrrred', 'idealware', 'caliendo', 'ashtonirwow', 'rightfooted', 'jbflexes', 'belivin', 'alidoee', 'skxidjjjxhhsjahxnja', 'secbig', 'beautifuuuuul', 'pourjeinventoryblowout', 'chieck', 'bowwowymcmb', 'zumbathon', 'dysert', 'yesto', 'reginaboy', 'this25', 'jetaime', 'unbealivable', 'prototypeversion', 'couldshould', 'fareal', 'goodasgold', 'quickaintfair', 'letsgo', 'gullit', 'instaclassic', 'dallascowboy', 'ballive', 'bigmeeks5', 'lhhatl', 'roubd', 'wusup', 'aauthorsmusic', 'newsmichael', 'chrissmoove', 'prayforkrista', 'jamiesalud1d', 'saramcmann', 'tripledouble', 'hollywoodcelebrity', 'solomonswisdom', 'devaneykk', '

In [26]:
len(missed)/len(vocab)

0.10952914798206279

In [27]:
train_data_missed = [(p, t) for p in clean_train_data for t in p.original + p.candidate if t in missed]

In [28]:
len(train_data_missed)/len(train_data)

0.3258456201214224

In [168]:
train_data_missed[:10]

[(Phrase(original=['person', 'the', 'number', 'qb', 'to', 'go', 'in', 'this', 'draft'], candidate=['wow', 'person', 'fsu', 'number', 'qb', 'taken'], label=True),
  'fsu'),
 (Phrase(original=['so', 'person', 'is', 'the', 'number', 'qb', 'taken'], candidate=['bills', 'take', 'sportsteam', 'number', 'qb', 'off', 'the', 'board'], label=True),
  'sportsteam'),
 (Phrase(original=['person', 'is', 'the', 'number', 'qb', 'in', 'the', 'draft'], candidate=['wow', 'person', 'fsu', 'number', 'qb', 'taken'], label=True),
  'fsu'),
 (Phrase(original=['person', 'was', 'the', 'number', 'qb', 'picked', 'overall'], candidate=['number', 'qb', 'taken', 'in', 'the', 'draft', 'go', 'noles'], label=True),
  'noles'),
 (Phrase(original=['person', 'was', 'the', 'number', 'qb', 'picked', 'overall'], candidate=['sportsteam', 's', 'person', 'was', 'the', 'number', 'qb', 'taken'], label=True),
  'sportsteam'),
 (Phrase(original=['person', 'was', 'the', 'number', 'qb', 'picked', 'overall'], candidate=['so', 'to', 'm

In [31]:
from keras.layers import Embedding

embedding_layer = Embedding(input_dim = VOCAB_SIZE, 
                            output_dim = EMBEDDING_SIZE, 
                            input_length = MAX_SEQ_LEN,
                            weights = [EMBEDDING_MATRIX], trainable = False)

In [32]:
from keras.layers import Input, LSTM, Embedding, TimeDistributed, Dense, Dropout
from keras.models import Model
from keras.optimizers import RMSprop

HIDDEN_DIM = 300

encoder_inputs = Input(shape=(MAX_SEQ_LEN, ), dtype='int32',)
encoder_embedding = embedding_layer(encoder_inputs)
encoder_LSTM = LSTM(HIDDEN_DIM, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)
    
decoder_inputs = Input(shape=(MAX_SEQ_LEN, ), dtype='int32',)
decoder_embedding = embedding_layer(decoder_inputs)
decoder_LSTM = LSTM(HIDDEN_DIM, return_state=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])
    
outputs = Dense(1, activation='sigmoid')(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], outputs)

model.summary()
model.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['acc'])

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 18)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 18)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 18, 300)      2676300     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(Non

In [33]:
model.fit([padding(train_encoder_seqs), padding(train_encoder_seqs)], np.array(train_labels),
          batch_size = 100, epochs = 5, validation_data=([padding(dev_encoder_seqs), padding(dev_encoder_seqs)], dev_labels))

Instructions for updating:
Use tf.cast instead.
Train on 11530 samples, validate on 4142 samples
Epoch 1/5
11530/11530 [==============================] - 12s 1ms/step - loss: 0.6459 - acc: 0.6517 - val_loss: 0.6469 - val_acc: 0.6451
Epoch 2/5
11530/11530 [==============================] - 10s 907us/step - loss: 0.6327 - acc: 0.6578 - val_loss: 0.6589 - val_acc: 0.6446
Epoch 3/5
11530/11530 [==============================] - 10s 873us/step - loss: 0.6140 - acc: 0.6690 - val_loss: 0.6918 - val_acc: 0.6470
Epoch 4/5
11530/11530 [==============================] - 10s 876us/step - loss: 0.5964 - acc: 0.6840 - val_loss: 0.6554 - val_acc: 0.6272
Epoch 5/5
11530/11530 [==============================] - 10s 875us/step - loss: 0.5879 - acc: 0.6879 - val_loss: 0.7008 - val_acc: 0.6420


In [34]:
from sklearn.metrics import classification_report

print(classification_report(dev_labels, [prob.item() > 0.5 for prob in model.predict([padding(dev_encoder_seqs), padding(dev_decoder_seqs)])]))

              precision    recall  f1-score   support

       False       0.65      0.98      0.78      2672
        True       0.44      0.03      0.06      1470

   micro avg       0.64      0.64      0.64      4142
   macro avg       0.54      0.50      0.42      4142
weighted avg       0.57      0.64      0.52      4142



In [35]:
from sklearn.metrics import classification_report

print(classification_report(test_labels, [prob.item() > 0.5 for prob in model.predict([padding(test_encoder_seqs), padding(test_decoder_seqs)])]))

              precision    recall  f1-score   support

       False       0.80      0.98      0.88       663
        True       0.42      0.05      0.08       175

   micro avg       0.79      0.79      0.79       838
   macro avg       0.61      0.51      0.48       838
weighted avg       0.72      0.79      0.71       838



In [62]:
def dump_scores(filename, predicted_similarity):
    with open(filename, 'w+') as f:
        for estimate in predicted_similarity:                    
            f.write("{}\t{:.4f}\n".format(str(estimate.item() > 0.5).lower(), estimate.item()))

In [63]:
dump_scores('PIT2015_zubovych_autoencoder.output', model.predict([padding(test_encoder_seqs), padding(test_decoder_seqs)]))

In [64]:
!python SemEval-PIT2015-py3/scripts/pit2015_eval_single.py SemEval-PIT2015-py3/data/test_bin.label PIT2015_zubovych_autoencoder.output

838	zubovych	autoencoder		F: 0.256	Prec: 0.398	Rec: 0.189		P-corr: 0.150	F1: 0.362	Prec: 0.241	Rec: 0.726


In [37]:
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Activation, Dropout
from keras.layers import Embedding, Input
from keras.layers import LSTM, Lambda, concatenate
from keras import regularizers
from keras.optimizers import Adadelta

In [511]:
HIDDEN_DIM=300

def exponent_neg_manhattan_distance(x, hidden_size=HIDDEN_DIM):
    ''' Helper function for the similarity estimate of the LSTMs outputs '''
    return K.exp(-K.sum(K.abs(x[:,:hidden_size] - x[:,hidden_size:]), axis=1, keepdims=True))

def exponent_neg_cosine_distance(x, hidden_size=HIDDEN_DIM):
    ''' Helper function for the similarity estimate of the LSTMs outputs '''
    leftNorm = K.l2_normalize(x[:,:hidden_size], axis=-1)
    rightNorm = K.l2_normalize(x[:,hidden_size:], axis=-1)
    return K.exp(K.sum(K.prod([leftNorm, rightNorm], axis=0), axis=1, keepdims=True))

In [169]:
seq_1 = Input(shape=(MAX_SEQ_LEN,), dtype='int32', name='sequence1')
seq_2 = Input(shape=(MAX_SEQ_LEN,), dtype='int32', name='sequence2')

input_1 = embedding_layer(seq_1)
input_2 = embedding_layer(seq_2)

l1 = LSTM(units=HIDDEN_DIM)

l1_out = l1(input_1)
l2_out = l1(input_2)

concats = concatenate([l1_out, l2_out], axis=-1)

#main_output = Lambda(exponent_neg_cosine_distance, output_shape=(1,))(concats)
#main_output = Lambda(exponent_neg_manhattan_distance, output_shape=(1,))(concats)
dense_ouput = Dense(1024, activation="relu")(concats)
main_output = Dense(1, activation="sigmoid")(dense_ouput)

model = Model(inputs=[seq_1, seq_2], outputs=[main_output])

opt = keras.optimizers.Adadelta(lr = 0.1, clipnorm=1.25)

#model.compile(optimizer=RMSprop(lr=1e-4), loss='mean_squared_error', metrics=['accuracy'])
#model.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=opt,loss='mean_squared_error', metrics=['accuracy'])
model.summary()

ValueError: "input_length" is 13, but received input has shape (None, 18)

In [46]:
n_hidden = 300
gradient_clipping_norm = 1.25
batch_size = 64
#n_epoch = 25

def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# The visible layer
left_input = Input(shape=(MAX_SEQ_LEN,), dtype='int32')
right_input = Input(shape=(MAX_SEQ_LEN,), dtype='int32')

#embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
model = Model([left_input, right_input], [malstm_distance])

# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta()#clipnorm=gradient_clipping_norm

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 18)           0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 18)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 18, 300)      2676300     input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
lstm_5 (LSTM)                   (None, 300)          721200      embedding_1[6][0]                
          

In [56]:
model.fit([padding(train_encoder_seqs), padding(train_decoder_seqs)], np.array(train_labels),
          batch_size = 64, epochs = 2, validation_data = ([padding(dev_encoder_seqs), padding(dev_decoder_seqs)], dev_labels))

Train on 11530 samples, validate on 4142 samples
Epoch 1/2
11530/11530 [==============================] - 13s 1ms/step - loss: 0.1789 - acc: 0.7414 - val_loss: 0.2176 - val_acc: 0.6569
Epoch 2/2
11530/11530 [==============================] - 13s 1ms/step - loss: 0.1755 - acc: 0.7475 - val_loss: 0.2645 - val_acc: 0.6543


In [65]:
from sklearn.metrics import classification_report

print(classification_report(dev_labels, [prob.item() > 0.1 for prob in model.predict([padding(dev_encoder_seqs), padding(dev_decoder_seqs)])]))

              precision    recall  f1-score   support

       False       0.73      0.55      0.62      2672
        True       0.43      0.63      0.51      1470

   micro avg       0.57      0.57      0.57      4142
   macro avg       0.58      0.59      0.57      4142
weighted avg       0.62      0.57      0.58      4142



In [64]:
from sklearn.metrics import classification_report

print(classification_report(test_labels, [prob.item() > 0.5 for prob in model.predict([padding(test_encoder_seqs), padding(test_decoder_seqs)])]))

              precision    recall  f1-score   support

       False       0.79      1.00      0.89       663
        True       0.80      0.02      0.04       175

   micro avg       0.79      0.79      0.79       838
   macro avg       0.80      0.51      0.46       838
weighted avg       0.80      0.79      0.71       838



In [61]:
[prob for prob in model.predict([padding(test_encoder_seqs), padding(test_decoder_seqs)])]

[array([0.19480483], dtype=float32),
 array([0.09724023], dtype=float32),
 array([0.1721062], dtype=float32),
 array([0.24150527], dtype=float32),
 array([0.01772683], dtype=float32),
 array([0.14233409], dtype=float32),
 array([0.03521702], dtype=float32),
 array([0.02362542], dtype=float32),
 array([0.05784016], dtype=float32),
 array([0.0470014], dtype=float32),
 array([0.05055037], dtype=float32),
 array([0.12591657], dtype=float32),
 array([0.02147567], dtype=float32),
 array([0.28779206], dtype=float32),
 array([0.09898105], dtype=float32),
 array([0.03990361], dtype=float32),
 array([0.08415841], dtype=float32),
 array([0.57341003], dtype=float32),
 array([0.10009886], dtype=float32),
 array([0.15213118], dtype=float32),
 array([0.32510757], dtype=float32),
 array([0.35704452], dtype=float32),
 array([0.19132434], dtype=float32),
 array([0.34612715], dtype=float32),
 array([0.02910237], dtype=float32),
 array([0.14084852], dtype=float32),
 array([0.05263627], dtype=float32),
 ar

In [51]:
dump_scores('PIT2015_zubovych_MaLSTM.output', model.predict([padding(test_encoder_seqs), padding(test_decoder_seqs)]))

In [52]:
!python SemEval-PIT2015-py3/scripts/pit2015_eval_single.py SemEval-PIT2015-py3/data/test_bin.label PIT2015_zubovych_MaLSTM.output

838	zubovych	MaLSTM		F: 0.332	Prec: 0.421	Rec: 0.274		P-corr: 0.257	F1: 0.392	Prec: 0.343	Rec: 0.457
